In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain_experimental.text_splitter import SemanticChunker
import chromadb
import ollama
import os
from tqdm import tqdm
from langchain_chroma import Chroma
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

C:\Users\AdityaKumarSingh\.conda\envs\yolov8\lib\site-packages\IPython\core\interactiveshell.py:3442: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
import os

In [3]:
path=r"D:/HRAgent/Policies"

In [4]:
fname=os.listdir(path)

In [5]:
# path=r"D:/HRAgent/Policies/ANTI-BRIBERY AND ANTI-CORRUPTION POLICY OF TATA INDUSTRIES.pdf"

In [6]:
path=r"D:/HRAgent/Policies"
fname=os.listdir(path)
resultantpdf=[]
for fl in fname:
    
    loader=PyPDFLoader(os.path.join(path,fl))
    resultantpdf.extend(loader.load())

In [7]:
# resultantpdf.extend(loader.load())

In [8]:
url="http://10.40.5.7:8000/v1"

In [9]:
import os
os.environ["OPENAI_API_KEY"]="None"

In [11]:
embedding=OpenAIEmbeddings(model="BAAI/bge-base-en-v1.5",base_url=url)

In [12]:
embedding.embed_query("Hi i am")

[-0.01018524169921875,
 0.030670166015625,
 0.037567138671875,
 0.0270538330078125,
 0.06915283203125,
 0.05084228515625,
 0.05157470703125,
 0.0286712646484375,
 -0.0167999267578125,
 -0.043212890625,
 0.0238037109375,
 0.0006937980651855469,
 -0.041656494140625,
 0.028045654296875,
 -0.018829345703125,
 0.062042236328125,
 0.067626953125,
 0.0038852691650390625,
 0.00348663330078125,
 0.018157958984375,
 0.032806396484375,
 0.00803375244140625,
 0.01091766357421875,
 -0.014190673828125,
 0.059967041015625,
 0.036956787109375,
 0.0231781005859375,
 0.0008215904235839844,
 -0.04541015625,
 -0.005035400390625,
 0.033966064453125,
 -0.0168914794921875,
 0.0026340484619140625,
 -0.033477783203125,
 -0.049957275390625,
 -0.039306640625,
 0.01131439208984375,
 -0.029693603515625,
 -0.01152801513671875,
 0.0065765380859375,
 -0.04327392578125,
 -0.0189666748046875,
 -0.01407623291015625,
 -0.003215789794921875,
 -0.0657958984375,
 -0.028045654296875,
 0.00673675537109375,
 -0.006786346435546

In [13]:
semanticchunker=SemanticChunker(embedding, breakpoint_threshold_type="percentile")

In [14]:
chunks=semanticchunker.create_documents([doc.page_content for doc in tqdm(resultantpdf[:5])])

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<?, ?it/s]


BadRequestError: Error code: 400 - {'object': 'error', 'message': 'Token id 74465 is out of vocabulary', 'type': 'BadRequestError', 'param': None, 'code': 400}

In [10]:
chroma_db = Chroma.from_documents(
    documents=chunks, 
    embedding=embedding, 
    persist_directory="knowledgebase2", 
    collection_name="llama_embed_knolwdge"
)

In [11]:
chroma_db.as_retriever().get_relevant_documents("finance vice persident of tata insights and quants")

C:\Users\AdityaKumarSingh\AppData\Local\Temp\ipykernel_12312\4032791689.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chroma_db.as_retriever().get_relevant_documents("finance vice persident of tata insights and quants")


[Document(metadata={}, page_content='3. In case of violations of this ABAC policy, the Compliance Officer  shall  take \nappropriate steps such as: \na) Assigning an Investigation Team : Experts with the right knowledge and \nobjectivity may be appointed to investigate a complaint. b) Conducting an Investigation : Every investigation  relating to a suspected \nviolation of this ABAC Policy shall  be investigated by the Compliance Officer  \ntogether with other members assigned under sub -clause (a) above. The \nobjective of such a n investigation would be to determine the facts, through \ninterviews with concerned participants and/or review of documents . Such \ninvestigation team will make a written demand for information, records etc .'),
 Document(metadata={}, page_content='Appropriate due diligence is conducted and properly documented; \nb. Formal commitment (in writing) is sought from the third party to ensure \ncompliance to these standards; '),
 Document(metadata={}, page_conten

In [12]:
prompt_template = """Use the following pieces of context to answer the question at the end. 
If you do not know the answer, please think rationally and answer from your own knowledge base.
Don't put extra information

{context}

Question: {question}
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [13]:
llm=ChatOllama(base_url=url,model="llama3.2:1b",temperature=0.1)

In [14]:
query="finance vp of tata iq?"

In [15]:
chain_type_kwargs = {"prompt": PROMPT}
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", 
    retriever=chroma_db.as_retriever(), 
    chain_type_kwargs=chain_type_kwargs)

qa_chain.run(query)

C:\Users\AdityaKumarSingh\AppData\Local\Temp\ipykernel_12312\2838688471.py:8: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa_chain.run(query)


'S Sriram is the Vice President (Finance) & Company Secretary.'

In [16]:
from pydantic import BaseModel
from langchain_core.pydantic_v1 import BaseModel, Field

C:\Users\AdityaKumarSingh\.conda\envs\yolov8\lib\site-packages\IPython\core\interactiveshell.py:3442: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

In [46]:
llm=ChatOllama(base_url="http://172.16.0.178:8010/",model="llama3.2:1b")

In [89]:
llm =OpenAI(base_url="http://10.40.0.18:8000/v1",model="meta-llama/Llama-3.2-1B")

In [95]:
class GradeDocument(BaseModel):
    binary_score: str= Field("Give a binary score 'yes' or 'no' score to indicate whether the document is relevant or not")

structure_llm_grader= llm.with_structured_output(GradeDocument)

NotImplementedError: 

In [ ]:
structure_llm_grader

In [ ]:
system=""" You are a grader assessing relevance of a retrieved document to a user question.\n
If the document contains keyword(s) or semantic meaning related to the question, grade it as a relevent.
Give a binary score 'yes' or 'no' score to indicate whether the document is relevant or not

"""

In [93]:
grade_prompt= ChatPromptTemplate.from_messages([
    ("system",system),
    ("human","""Retrieved documents: \n\n {documents} \n\n user_question: {question}""")
])

In [94]:
doc=chroma_db.as_retriever().get_relevant_documents(query)

NameError: name 'chroma_db' is not defined

In [ ]:
retrieval_grader=grade_prompt|structure_llm_grader

In [66]:
retrieval_grader.invoke({"question":query,"documents":doc[0].page_content})

ReadError: [WinError 10053] An established connection was aborted by the software in your host machine

In [59]:
retrieval_grader.get_graph()

Graph(nodes={'ed94f93e16b7484a861a2bad4dd2ff79': Node(id='ed94f93e16b7484a861a2bad4dd2ff79', name='PromptInput', data=<class 'langchain_core.utils.pydantic.PromptInput'>, metadata=None), 'bbd50f01c04b43c8aa4ee068c3d481aa': Node(id='bbd50f01c04b43c8aa4ee068c3d481aa', name='ChatPromptTemplate', data=ChatPromptTemplate(input_variables=['documents', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template=" You are a grader assessing relevance of a retrieved document to a user question.\n\nIf the document contains keyword(s) or semantic meaning related to the question, grade it as a relevent.\nGive a binary score 'yes' or 'no' score to indicate whether the document is relevant or not\n\n"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['documents', 'question'], input_types={}, partial_variables={}, template='Retrieved documents:

In [48]:
url="http://10.40.0.18:8000/v1"

In [53]:
ai=OpenAI(base_url=url,model="meta-llama/Llama-3.2-3B-Instruct")

In [60]:
ai.invoke("asdbjkjk")

'\n\nNO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO. NO'